Benjamin Ye  
CS/CNE/EE 156a: Learning Systems (Fall 2023)  
October 9, 2023

## Homework 2

In [1]:
import sys

import numpy as np

sys.path.insert(0, str(globals()['_dh'][0].resolve().parent))
from cs156a import (coin_flip, hoeffding_inequality, 
                    target_function_random_line, perceptron, linear_regression,
                    target_function_hw2, generate_data, validate_binary)

#### Problems 1–2

In [2]:
rng = np.random.default_rng()
n_trials = 100_000
n_flips = 10
labels = ("first", "random", "minimum")
print(f"\n[HW2 P1–2]\nCoin flip statistics over {n_trials:,} trials:")
nus = coin_flip(n_trials, rng=rng)
for label, nu in zip(labels, nus.mean(axis=1)):
    print(f"  {label}: {nu=:.5f}")

print("\nHoeffding's inequality:")
epss = np.linspace(0, 0.5, 6)
hist = np.apply_along_axis(
    lambda x: np.histogram(x, bins=np.linspace(-0.05, 1.05, 12))[0], 1, nus
) # requires at least 8 GB RAM
probs = np.hstack((hist[:, (5,)], hist[:, 4::-1] + hist[:, 6:])) / n_trials
bounds = hoeffding_inequality(n_flips, epss)
print("   eps |  bound  | ", " | ".join(l.center(15) for l in labels),
      "\n  -----+---------+", "+".join(3 * [17 * "-"]), sep="")
for eps, bound, prob, satisfy in zip(
        epss, bounds, probs.T, (probs <= bounds).T):
    print(
        f"   {eps:.1f} | {bound:.5f} |",
        " | ".join(
            f"{p:.5f} ({s})".ljust(15) for p, s in zip(prob, satisfy)
        )
    )


[HW2 P1–2]
Coin flip statistics over 100,000 trials:
  first: nu=0.50022
  random: nu=0.49990
  minimum: nu=0.03773

Hoeffding's inequality:
   eps |  bound  |      first      |      random     |     minimum    
  -----+---------+-----------------+-----------------+-----------------
   0.0 | 2.00000 | 0.24503 (True)  | 0.24671 (True)  | 0.00000 (True) 
   0.1 | 1.63746 | 0.40849 (True)  | 0.41019 (True)  | 0.00000 (True) 
   0.2 | 0.89866 | 0.23619 (True)  | 0.23471 (True)  | 0.00000 (True) 
   0.3 | 0.33060 | 0.08894 (True)  | 0.08714 (True)  | 0.00002 (True) 
   0.4 | 0.08152 | 0.01934 (True)  | 0.01900 (True)  | 0.37724 (False)
   0.5 | 0.01348 | 0.00201 (True)  | 0.00225 (True)  | 0.62274 (False)


#### Problems 5–7

In [3]:
N = 100
n_runs = 1_000
print(f"\n[HW2 P5–7]\nLinear regression statistics over {n_runs:,} runs:")
E_in, E_out = np.mean(
    [linear_regression(N, target_function_random_line(rng=rng), 
                       validate_binary, rng=rng)
     for _ in range(n_runs)], 
    axis=0
)
print(f"  {N=:,}, {E_in=:.3f}, {E_out=:.3f}")

N = 10
print("\nPLA (with linear regression hypothesis) statistics over",
      f"{n_runs:,} runs:")
iters = np.empty(n_runs, dtype=float)
for i in range(n_runs):
    f = target_function_random_line(rng=rng)
    x, y = generate_data(N, f, bias=True, rng=rng)
    iters[i] = perceptron(
        N, f, validate_binary, x=x, y=y, rng=rng,
        w=linear_regression(N, f, validate_binary, x=x, y=y, rng=rng, 
                            hyp=True)[0]
    )[0]
print(f"  {N=:,}, iters={iters.mean():,.0f}")


[HW2 P5–7]
Linear regression statistics over 1,000 runs:
  N=100, E_in=0.041, E_out=0.049

PLA (with linear regression hypothesis) statistics over 1,000 runs:
  N=10, iters=4


#### Problems 8–10

In [4]:
f = target_function_hw2()
N = N_test = n_runs = 1_000
noise = (0.1, lambda y: -y)
print("\n[HW2 P8–10]\nLinear regression (with linear feature vector)",
      f"statistics over {n_runs:,} runs:")
E_in = np.mean(
    [linear_regression(N, f, validate_binary, noise=noise, rng=rng)[0]
     for _ in range(n_runs)]
)
print(f"  {N=:,}, noise={noise[0]:.3f}, {E_in=:.3f}")

transform = lambda x: np.hstack((
    x, 
    x[:, 1:2] * x[:, 2:], 
    x[:, 1:2] ** 2,
    x[:, 2:] ** 2
))
gs = np.array(((-1, -0.05, 0.08, 0.13, 1.5, 1.5), 
               (-1, -0.05, 0.08, 0.13, 1.5, 15),
               (-1, -0.05, 0.08, 0.13, 15, 1.5),
               (-1, -1.5, 0.08, 0.13, 0.05, 0.05),
               (-1, -0.05, 0.08, 1.5, 0.15, 0.15)))
print("\nLinear regression (with nonlinear feature vector) hypothesis",
      f"over {n_runs:,} runs:")
w = np.mean(
    [linear_regression(N, f, validate_binary, transform=transform, 
                       noise=noise, rng=rng, hyp=True)[0]
     for _ in range(n_runs)],
    axis=0
)
print("  w=[", ", ".join(f"{v:.5f}" for v in w), "]", sep="")
probs = np.zeros((N_test, 5))
Es_out = np.zeros(N_test)
for i in range(n_runs):
    x_test, y_test = generate_data(N_test, f, bias=True, rng=rng)
    x_test = transform(x_test)
    y_test[rng.choice(N_test, round(noise[0] * N_test), False)] *= -1
    h_test = np.sign(x_test @ w)
    probs[i] = validate_binary(gs.T, x_test, h_test[:, None])
    Es_out[i] = np.count_nonzero(h_test != y_test) / N_test
for i, (g, p) in enumerate(zip(gs, probs.mean(axis=0))):
    print(f"  [{chr(97 + i)}] g=[", ", ".join(f"{v:.2g}" for v in g),
          f"] (prob={1 - p:.5f})", sep="")
print(f"  {N=:,}, noise={noise[0]:.3f}, E_out={Es_out.mean():.3f}")


[HW2 P8–10]
Linear regression (with linear feature vector) statistics over 1,000 runs:
  N=1,000, noise=0.100, E_in=0.504

Linear regression (with nonlinear feature vector) hypothesis over 1,000 runs:
  w=[-0.99449, -0.00138, 0.00158, 0.00036, 1.55987, 1.56042]
  [a] g=[-1, -0.05, 0.08, 0.13, 1.5, 1.5] (prob=0.97216)
  [b] g=[-1, -0.05, 0.08, 0.13, 1.5, 15] (prob=0.66315)
  [c] g=[-1, -0.05, 0.08, 0.13, 15, 1.5] (prob=0.66256)
  [d] g=[-1, -1.5, 0.08, 0.13, 0.05, 0.05] (prob=0.63276)
  [e] g=[-1, -0.05, 0.08, 1.5, 0.15, 0.15] (prob=0.56092)
  N=1,000, noise=0.100, E_out=0.123
